I had a bit of free time this holiday season, so of course I spent some of it getting caught up on the Veratasim channel on YouTube. Unfortunately, this resulted in second-degree burns from the most dangerous problem in mathematics: 3x+1, also known as the Collatz conjecture.

In the video, there is a segment a few seconds long that shows the binary representation of the sequence of numbers as an image generated in matlab.

https://youtu.be/094y1Z2wpJg?t=1056

(Watch the whole video if you have not heard of this conjecture before, Derek does a great job explaining it.)

Another way of saying the conjecture is that if you multiply a number by three and then add 1 to the least significant digit repeatedly, the number of significant digits will always converge to 1.

I'm using this form below, because I think it helps to demonstrate the dynamics of the problem more intuitively. In this arrangement, you can see how information can only flow from right to left.

In [8]:
# Let's use python, it has arbitrary precision integers.


def power(x):
    for i in range(len(f"{x:b}")):
        if x % (2 ** (i + 1)):
            return i


def step(x):
    return 3 * x + (1 << power(x))


def print_line(x, width):
    digits = f"{x:b}".replace("0", " ").rjust(width, " ")
    print(f"{digits} | {x >> power(x)}")

# Quick-and-dirty function to generate some lines of text
def print_graph(x, width):
    # the conjecture says that this loop will terminate
    while len(f"{x:b}".strip("0")) > 1:
        print_line(x, width)
        x = step(x)

    # iterate a few more times for good measure
    for _ in range(5):
        print_line(x, width)
        x = step(x)


print_graph((1 << 20) + 1, 80)

                                                                              1                    1 | 2097153
                                                                             11                  1   | 1572865
                                                                           1  1                1     | 1179649
                                                                          11 11              1       | 884737
                                                                        1 1   1            1         | 663553
                                                                       1111  11          1           | 497665
                                                                     1 11 11  1        1             | 373249
                                                                   1   1   1 11      1               | 279937
                                                                  11  11 1    1    1                 | 209953
       

Let's make some observations. 

- The significance of the seed 1048577 is completely obfuscated in base-10, but in base-2 we see why this was chosen. We want to initially isolate the dynamics of the left and right edges.
- The bit on the right edge right initially moves left at 4x, it is already in the equivalent of the 1, 4, 2, 1 loop.
- The left edge of the graph is growing at a rate of 3x.
- Towards the bottom left of the graph, the edges converge to into the stable and familiar 4x or 1, 4, 2, 1 loop.
- On occasion, the right edge fails to move left at 4x. If the two least significant digits on the right are equal to 11 in binary (3 in decimal), then the right edge will move left only at 2x. Otherwise, it will move left at least 4x, and sometimes much more.

So, why is this not enough to prove the conjecture? If the left side is mostly growing slower than the right side, won't the edges eventually converge?

I'll answer that with a different example.

In [6]:
print_graph((1 << 15) - 1, 100)

                                                                                     111111111111111 | 32767
                                                                                   1 11111111111111  | 49151
                                                                                 1   1111111111111   | 73727
                                                                                11 1 111111111111    | 110591
                                                                              1 1    11111111111     | 165887
                                                                             1111  1 1111111111      | 248831
                                                                           1 11 11   111111111       | 373247
                                                                         1   1   1 1 11111111        | 559871
                                                                        11  11 1     1111111         | 839807
             

The number 32767 starts orders of magnitude smaller, but takes more iterations to stabilize. This is mostly because we give it a solid block of 1's as a seed value.

Some more observations:
- For the first set of iterations, The right side is growing at 3x as always.
- The left side cannot grow faster than 2x until it gets past that initial block. The edges diverge (and the equivalent number grows) during this time.
- Blocks can form naturally: halfway down from the top, another block appears right on the edge. Once again, the edges diverge.

If there were some initial seed that could make the two rightmost bits equal to 11 in binary most of the time, the number might escape the conjecture's gravity and grow on forever.

I hope this little experiment helps show why it is so hard to prove the conjecture. The pseudorandom pattern of bits interacting caused by multiplying by three seems to be non-repeating, just like log2(3) is an irrational number.

From where I am sitting, it's impossible to say that there is not some seed number that manages to spew out just the right combination of bits over and over to keep itself growing and growing.